In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindi-english-dataset/Sentence pairs in English-Hindi - 2025-02-11.tsv


In [2]:
!pip3 install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
from indicnlp.tokenize import indic_tokenize

In [4]:
data = pd.read_csv("/kaggle/input/hindi-english-dataset/Sentence pairs in English-Hindi - 2025-02-11.tsv", sep="\t", header=None, names=["SrcSentID","SrcSent","DstSentID","DstSent"])

In [5]:
data.head()

,SrcSentID,SrcSent,DstSentID,DstSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।


In [6]:
data.shape

(13182, 4)

In [7]:
data.drop(labels=[data.columns[0],data.columns[2]],axis=1,inplace=True)

In [8]:
data.head()

,SrcSent,DstSent
0,Muiriel is 20 now.,म्यूरियल अब बीस साल की हो गई है।
1,Muiriel is 20 now.,म्यूरियल अब बीस साल की है।
2,Education in this world disappoints me.,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,That won't happen.,वैसा नहीं होगा।
4,I miss you.,मुझें तुम्हारी याद आ रही है।


In [9]:
src_sent_tokenizer = AutoTokenizer.from_pretrained("google-T5/T5-base")

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
data["SrcSent"] = data["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))

In [11]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की हो गई है।
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की है।
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...",मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,"[▁That, ▁won, ', t, ▁happen, .]",वैसा नहीं होगा।
4,"[▁I, ▁miss, ▁you, .]",मुझें तुम्हारी याद आ रही है।


In [12]:
data["DstSent"] = data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x, lang="hi"))

In [13]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[वैसा, नहीं, होगा, ।]"
4,"[▁I, ▁miss, ▁you, .]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [14]:
data["SrcSent"] = data["SrcSent"].apply(src_sent_tokenizer.convert_tokens_to_ids)

In [15]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[466, 751, 31, 17, 1837, 5]","[वैसा, नहीं, होगा, ।]"
4,"[27, 3041, 25, 5]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [16]:
Vs = src_sent_tokenizer.get_vocab()

In [17]:
Vs

{'▁disponibile': 17672,
 '▁Bundesliga': 28117,
 '▁tablespoon': 18396,
 'cili': 13067,
 'adia': 17327,
 'sie': 2452,
 '▁Medien': 12755,
 '▁dezvolt': 11464,
 '▁closed': 3168,
 '▁sehr': 1319,
 'May': 15881,
 '▁tablets': 11676,
 '▁Poli': 6907,
 'richtung': 18055,
 'Est': 14997,
 '▁bean': 15431,
 '▁To': 304,
 '▁Rap': 15052,
 '▁intoarce': 31266,
 '▁verification': 17549,
 '▁emoţi': 31487,
 '▁Tradition': 16418,
 '▁Geo': 10107,
 '▁dealers': 15939,
 '▁extension': 4924,
 'storm': 21556,
 'BAC': 26953,
 '▁tir': 11857,
 '▁représentant': 26851,
 '▁fleur': 14035,
 'Just': 19969,
 '▁Titan': 13622,
 '▁Zhang': 28015,
 '▁Resource': 16154,
 'traditionellen': 31341,
 '▁strategically': 25178,
 '▁texts': 14877,
 '▁chairman': 13404,
 '▁Constanța': 31526,
 'INCLUD': 27450,
 '▁roads': 7540,
 'divider': 31107,
 '▁folded': 23166,
 '▁group': 563,
 '▁Betroffenen': 30768,
 '▁Eminescu': 31174,
 '▁Sep': 12219,
 'deemed': 10863,
 'frac': 9880,
 'FORM': 24030,
 '▁Planning': 9557,
 '▁undertaken': 18046,
 'hopped': 29074,

In [18]:
len(Vs)

32100

In [19]:
hindi_vocab = set()
for tokenized_hindi_sent in data["DstSent"]:
    hindi_vocab.update(tokenized_hindi_sent)

In [20]:
hindi_vocab

{'ब्रायन',
 'मजबूरी',
 '।',
 'सोमवार',
 'फ़ैक्टरी',
 'विविधता',
 'पाउंड',
 'प्रति',
 '७११',
 'सस्ते',
 'मीट',
 'तोहफ़े',
 'बूढ़ा',
 'वीरान',
 'माफ़ी',
 'पछाड़',
 'माँ',
 'पीजिए',
 'तम्हे',
 'टेबल',
 'स्कूल',
 'शर्म',
 'गायिका',
 'जिनसे',
 'कर्तव्य',
 'ग़ुलाम',
 'पहुँचने',
 'कलात्मक',
 'सुरियाह',
 'जाये',
 'चौंका',
 'ची',
 'डरा',
 'ईमानदार',
 'जातें',
 'कमाई',
 'मुख्य',
 'विद्यार्थियों',
 'भिखारियों',
 'घोड़ा',
 'चाहिए',
 'गुस्से',
 'बीच',
 'पसंद',
 'माईने',
 'साइकिल',
 'उतारी',
 'दादीजी',
 'बाया',
 'चूहे',
 'प्रयास',
 'खत',
 'ढुँढ',
 'पराग्वे',
 'जल्दी',
 'कार्बनिक',
 'लाईन',
 'दिवस',
 'लेट',
 'पैंसठ',
 'कठोर',
 'तीसरी',
 'हमेषा',
 'दादी',
 'कांच',
 'कॉफी',
 'असभ्य',
 'शाखाएं',
 'सब्ज़ीयाँ',
 'खटखटाने',
 'भूलना',
 'थी',
 'उठाया',
 'तैर',
 'कूद',
 'जबरदस्ती',
 'श्वार्झनेगर',
 'जनतंत्र',
 '१२',
 'ख़रीद',
 'अनपेक्षित',
 'खाइए',
 'कूदो',
 'चेतावनी',
 'युद्ध',
 'वकील',
 'कहीं',
 'चुन',
 'इंग्लिश',
 'करो',
 'देशप्रेमी',
 'सिखाएंगे',
 'समझिए',
 'देखना',
 'काबुकी',
 'हरी',
 'बार्सेलोना',
 'जुल

In [21]:
vd = dict()
for idx, token in enumerate(hindi_vocab):
    vd[token] = idx + 3
vd["<PAD>"] = 0
vd["<SOS>"] = 1
vd["<EOS>"] = 2

In [22]:
print(vd)

{'ब्रायन': 3, 'मजबूरी': 4, '।': 5, 'सोमवार': 6, 'फ़ैक्टरी': 7, 'विविधता': 8, 'पाउंड': 9, 'प्रति': 10, '७११': 11, 'सस्ते': 12, 'मीट': 13, 'तोहफ़े': 14, 'बूढ़ा': 15, 'वीरान': 16, 'माफ़ी': 17, 'पछाड़': 18, 'माँ': 19, 'पीजिए': 20, 'तम्हे': 21, 'टेबल': 22, 'स्कूल': 23, 'शर्म': 24, 'गायिका': 25, 'जिनसे': 26, 'कर्तव्य': 27, 'ग़ुलाम': 28, 'पहुँचने': 29, 'कलात्मक': 30, 'सुरियाह': 31, 'जाये': 32, 'चौंका': 33, 'ची': 34, 'डरा': 35, 'ईमानदार': 36, 'जातें': 37, 'कमाई': 38, 'मुख्य': 39, 'विद्यार्थियों': 40, 'भिखारियों': 41, 'घोड़ा': 42, 'चाहिए': 43, 'गुस्से': 44, 'बीच': 45, 'पसंद': 46, 'माईने': 47, 'साइकिल': 48, 'उतारी': 49, 'दादीजी': 50, 'बाया': 51, 'चूहे': 52, 'प्रयास': 53, 'खत': 54, 'ढुँढ': 55, 'पराग्वे': 56, 'जल्दी': 57, 'कार्बनिक': 58, 'लाईन': 59, 'दिवस': 60, 'लेट': 61, 'पैंसठ': 62, 'कठोर': 63, 'तीसरी': 64, 'हमेषा': 65, 'दादी': 66, 'कांच': 67, 'कॉफी': 68, 'असभ्य': 69, 'शाखाएं': 70, 'सब्ज़ीयाँ': 71, 'खटखटाने': 72, 'भूलना': 73, 'थी': 74, 'उठाया': 75, 'तैर': 76, 'कूद': 77, 'जबरदस्ती': 78, 'श्वार्झन

In [23]:
def convert_hindi_tokens_to_ids(hindi_sent):
    return [vd[token] for token in hindi_sent]

In [24]:
data["DstSent"] = data["DstSent"].apply(lambda x: convert_hindi_tokens_to_ids(x))

In [25]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[2041, 4592, 2259, 2711, 6842, 2169, 3233, 227..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[2041, 4592, 2259, 2711, 6842, 2277, 5]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[6954, 5102, 2775, 3456, 5688, 3492, 3533, 665..."
3,"[466, 751, 31, 17, 1837, 5]","[6554, 2829, 3467, 5]"
4,"[27, 3041, 25, 5]","[2328, 255, 3912, 1323, 4870, 2277, 5]"


In [26]:
def insert_sos_token_id(hindi_sent_token_ids):
    return [1] + hindi_sent_token_ids

In [27]:
data["DstSentInput"] = data["DstSent"].apply(lambda x: insert_sos_token_id(x))

In [28]:
def insert_eos_token_id(hindi_sent_token_ids):
    return hindi_sent_token_ids + [2]

In [29]:
data["DstSentLabel"] = data["DstSent"].apply(lambda x: insert_eos_token_id(x))

In [30]:
data.head()

,SrcSent,DstSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[2041, 4592, 2259, 2711, 6842, 2169, 3233, 227...","[1, 2041, 4592, 2259, 2711, 6842, 2169, 3233, ...","[2041, 4592, 2259, 2711, 6842, 2169, 3233, 227..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[2041, 4592, 2259, 2711, 6842, 2277, 5]","[1, 2041, 4592, 2259, 2711, 6842, 2277, 5]","[2041, 4592, 2259, 2711, 6842, 2277, 5, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[6954, 5102, 2775, 3456, 5688, 3492, 3533, 665...","[1, 6954, 5102, 2775, 3456, 5688, 3492, 3533, ...","[6954, 5102, 2775, 3456, 5688, 3492, 3533, 665..."
3,"[466, 751, 31, 17, 1837, 5]","[6554, 2829, 3467, 5]","[1, 6554, 2829, 3467, 5]","[6554, 2829, 3467, 5, 2]"
4,"[27, 3041, 25, 5]","[2328, 255, 3912, 1323, 4870, 2277, 5]","[1, 2328, 255, 3912, 1323, 4870, 2277, 5]","[2328, 255, 3912, 1323, 4870, 2277, 5, 2]"


In [31]:
data.drop(labels=[data.columns[1]], axis=1, inplace=True)

In [32]:
data.head()

,SrcSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 2041, 4592, 2259, 2711, 6842, 2169, 3233, ...","[2041, 4592, 2259, 2711, 6842, 2169, 3233, 227..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 2041, 4592, 2259, 2711, 6842, 2277, 5]","[2041, 4592, 2259, 2711, 6842, 2277, 5, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1, 6954, 5102, 2775, 3456, 5688, 3492, 3533, ...","[6954, 5102, 2775, 3456, 5688, 3492, 3533, 665..."
3,"[466, 751, 31, 17, 1837, 5]","[1, 6554, 2829, 3467, 5]","[6554, 2829, 3467, 5, 2]"
4,"[27, 3041, 25, 5]","[1, 2328, 255, 3912, 1323, 4870, 2277, 5]","[2328, 255, 3912, 1323, 4870, 2277, 5, 2]"


In [33]:
x = list(data["SrcSent"])